# Classify Flowers Using Measurements

This example is from: https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/

The test problem we will be using in this tutorial is iris classification.

Based on the sepal length, sepal width, petal length and petal width of 150 observations of iris flowers, we need to implement a classification model to predict the species, which is one of setosa, versicolor or virginica.

It is a standard dataset where the species is known for all instances. As such we can split the data into training and test datasets and use the results to evaluate our algorithm implementation. Good classification accuracy on this problem is above 90% correct, typically 96% or better.

Here is a iris flower image:
<img src="iris_petal_sepal.png" />

# How to implement k-Nearest Neighbors in Python

This tutorial is broken down into the following steps:

1. Handle Data: Open the dataset from CSV and split into test/train datasets.
2. Similarity: Calculate the distance between two data instances.
3. Neighbors: Locate k most similar data instances.
4. Response: Generate a response from a set of data instances.
5. Accuracy: Summarize the accuracy of predictions.
6. Main: Tie it all together.


## 1. Handle Data

The first thing we need to do is load our data file. The data is in CSV format without a header line or any quotes. We can open the file with the open function and read the data lines using the reader function in the csv module.

In [2]:
import csv
with open('iris.data', 'r') as csvfile:
	lines = csv.reader(csvfile)
	for row in lines:
		print(row)

['5.1', '3.5', '1.4', '0.2', 'Iris-setosa']
['4.9', '3.0', '1.4', '0.2', 'Iris-setosa']
['4.7', '3.2', '1.3', '0.2', 'Iris-setosa']
['4.6', '3.1', '1.5', '0.2', 'Iris-setosa']
['5.0', '3.6', '1.4', '0.2', 'Iris-setosa']
['5.4', '3.9', '1.7', '0.4', 'Iris-setosa']
['4.6', '3.4', '1.4', '0.3', 'Iris-setosa']
['5.0', '3.4', '1.5', '0.2', 'Iris-setosa']
['4.4', '2.9', '1.4', '0.2', 'Iris-setosa']
['4.9', '3.1', '1.5', '0.1', 'Iris-setosa']
['5.4', '3.7', '1.5', '0.2', 'Iris-setosa']
['4.8', '3.4', '1.6', '0.2', 'Iris-setosa']
['4.8', '3.0', '1.4', '0.1', 'Iris-setosa']
['4.3', '3.0', '1.1', '0.1', 'Iris-setosa']
['5.8', '4.0', '1.2', '0.2', 'Iris-setosa']
['5.7', '4.4', '1.5', '0.4', 'Iris-setosa']
['5.4', '3.9', '1.3', '0.4', 'Iris-setosa']
['5.1', '3.5', '1.4', '0.3', 'Iris-setosa']
['5.7', '3.8', '1.7', '0.3', 'Iris-setosa']
['5.1', '3.8', '1.5', '0.3', 'Iris-setosa']
['5.4', '3.4', '1.7', '0.2', 'Iris-setosa']
['5.1', '3.7', '1.5', '0.4', 'Iris-setosa']
['4.6', '3.6', '1.0', '0.2', 'Ir

Next we need to split the data into a training dataset that kNN can use to make predictions and a test dataset that we can use to evaluate the accuracy of the model.

We first need to convert the flower measures that were loaded as strings into numbers that we can work with. Next we need to split the data set randomly into train and datasets. A ratio of 67/33 for train/test is a standard ratio used.

Pulling it all together, we can define a function called loadDataset that loads a CSV with the provided filename and splits it randomly into train and test datasets using the provided split ratio.

In [3]:
import csv
import random
def loadDataset(filename, split, trainingSet=[] , testSet=[]):
	with open(filename, 'r') as csvfile:
	    lines = csv.reader(csvfile)
	    dataset = list(lines)
        # loop through each row
	    for x in range(len(dataset)-1):
	        for y in range(4):
	            dataset[x][y] = float(dataset[x][y])
                # random.random() generate a number [0,1)
	        if random.random() < split:
	            trainingSet.append(dataset[x])
	        else:
	            testSet.append(dataset[x])

We can test this function out with our iris dataset, as follows:

In [13]:
trainingSet=[]
testSet=[]
loadDataset('iris.data', 0.66, trainingSet, testSet)
print(random.random())
# repr() Return a string containing a printable representation
print('Train: ' + repr(len(trainingSet)))
print('Test: ' + repr(len(testSet)))

0.8598342756989173
Train: 102
Test: 48


## 2. Similarity

In order to make predictions we need to calculate the similarity between any two given data instances. This is needed so that we can locate the k most similar data instances in the training dataset for a given member of the test dataset and in turn make a prediction.

Given that all four flower measurements are numeric and have the same units, we can directly use the Euclidean distance measure. This is defined as the square root of the sum of the squared differences between the two arrays of numbers (read that again a few times and let it sink in).

Additionally, we want to control which fields to include in the distance calculation. Specifically, we only want to include the first 4 attributes. One approach is to limit the euclidean distance to a fixed length, ignoring the final dimension.

Putting all of this together we can define the euclideanDistance function as follows:

In [15]:
import math
def euclideanDistance(instance1, instance2, length):
	distance = 0
	for x in range(length):
		distance += pow((instance1[x] - instance2[x]), 2)
	return math.sqrt(distance)

In [ ]:
We can test this function with some sample data, as follows:

In [18]:
data1 = [2, 2, 2, 2, 'a']
data2 = [4, 4, 4, 4, 'b']
distance = euclideanDistance(data1, data2, 4)
print('Distance: ' + repr(distance))

Distance: 4.0


## 3. Neighbors

Now that we have a similarity measure, we can use it collect the k most similar instances for a given unseen instance.

This is a straight forward process of calculating the distance for all instances and selecting a subset with the smallest distance values.

Below is the getNeighbors function that returns k most similar neighbors from the training set for a given test instance (using the already defined euclideanDistance function)

In [34]:
import operator 
def getNeighbors(trainingSet, testInstance, k):
	distances = []
	length = len(testInstance)-1
	for x in range(len(trainingSet)):
		dist = euclideanDistance(testInstance, trainingSet[x], length)
		distances.append((trainingSet[x], dist))
    # key=operator.itemgetter(1) => order distances list based on second column (1)
	distances.sort(key=operator.itemgetter(1))
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x][0])
	return neighbors

We can test out this function as follows:

In [35]:
trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5, 'b']
k = 1
neighbors = getNeighbors(trainSet, testInstance, k)
print(neighbors)

[[4, 4, 4, 'b']]


## 4. Response

Once we have located the most similar neighbors for a test instance, the next task is to devise a predicted response based on those neighbors.

We can do this by allowing each neighbor to vote for their class attribute, and take the majority vote as the prediction.

Below provides a function for getting the majority voted response from a number of neighbors. It assumes the class is the last attribute for each neighbor.

In [37]:
import operator
def getResponse(neighbors):
    # classVotes is a dict, similar to JSON
    # e.g., {'a': 1, 'b': 2}
	classVotes = {}
	for x in range(len(neighbors)):
        # based on the last element
		response = neighbors[x][-1]
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
    # reverse=True means the most voted item will be our response
    # itemgetter(1): get the first item, i.e. item with most votes
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes[0][0]

We can test out this function with some test neighbors, as follows:

In [38]:
neighbors = [[1,1,1,1,'a'], [2,2,2,2,'a'], [3,3,3,3,'b']]
response = getResponse(neighbors)
print(response)

b


This approach returns one response in the case of a draw, but you could handle such cases in a specific way, such as returning no response or selecting an unbiased random response.

## 5. Accuracy

We have all of the pieces of the kNN algorithm in place. An important remaining concern is how to evaluate the accuracy of predictions.

An easy way to evaluate the accuracy of the model is to calculate a ratio of the total correct predictions out of all predictions made, called the classification accuracy.

Below is the getAccuracy function that sums the total correct predictions and returns the accuracy as a percentage of correct classifications.

In [39]:
def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] is predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

We can test this function with a test dataset and predictions, as follows:

In [24]:
testSet = [[1,1,1,1,'a'], [2,2,2,2,'a'], [3,3,3,3,'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


## 6. Main

We now have all the elements of the algorithm and we can tie them together with a main function.

Below is the example of implementing the kNN algorithm from scratch in Python.

Group Activity: You need to finish the TODO section.

In [3]:
pip install sklearn

You should consider upgrading via the '/home/jacob/ml_projects/ml_project1_env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Example of kNN implemented from Scratch in Python

import csv
import random
import math
import operator
from sklearn.neighbors import KNeighborsClassifier

def loadDataset(filename, split, trainingSet=[] , testSet=[]):
	with open(filename, 'r') as csvfile:
	    lines = csv.reader(csvfile)
	    dataset = list(lines)
	    for x in range(len(dataset)-1):
	        for y in range(4):
	            dataset[x][y] = float(dataset[x][y])
	        if random.random() < split:
	            trainingSet.append(dataset[x])
	        else:
	            testSet.append(dataset[x])

def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0
	
    
def kNeighbors(trainingSet, testSet, k):
    cl = KNeighborsClassifier(n_neighbors=k)
    features = []
    label = []
    
    for x in range(len(trainingSet)):
        features.append(trainingSet[x][:3]) #Create the list of features for each data value
        label.append(trainingSet[x][4]) #Create the labels for the corresponding data
        
    cl.fit(features, label) #Train Model on Training Set
    
    predictions = []
    
    for x in range(len(testSet)):
        predictions.append(cl.predict([testSet[x][0:3]]))
    
    return predictions

def main():

	repetitions = 5
	numK = 20
	maxAcc, maxK = 0, 0
	for y in range (numK):
		k = y+1
		sumOfAcc = 0 
		print('k= ' + repr(k))
        
		for x in range(repetitions):
			# prepare data
			trainingSet=[]
			testSet=[]
			split = 0.67
			loadDataset('iris.data', split, trainingSet, testSet)
			# generate predictions
			predictions=[]

			#Sklearn Classifier Code
			predictions = kNeighbors(trainingSet, testSet, k)
    
			accuracy = getAccuracy(testSet, predictions)
			sumOfAcc = sumOfAcc+accuracy
			print('Accuracy: ' + repr(accuracy) + '%')
    
		print('Average Accuracy:' + repr(sumOfAcc / repetitions) + '%')

		if maxAcc < (sumOfAcc / repetitions):
			maxAcc = (sumOfAcc / repetitions)
			maxK = k
            
	print("\nMost Accurate K: " + repr(maxK))
	print(maxAcc)
main()

k= 1
Accuracy: 95.65217391304348%
Accuracy: 95.83333333333334%
Accuracy: 91.66666666666666%
Accuracy: 89.28571428571429%
Accuracy: 93.75%
Average Accuracy:93.23757763975155%
k= 2
Accuracy: 94.91525423728814%
Accuracy: 95.0%
Accuracy: 94.11764705882352%
Accuracy: 88.37209302325581%
Accuracy: 95.65217391304348%
Average Accuracy:93.6114336464822%
k= 3
Accuracy: 96.0%
Accuracy: 96.36363636363636%
Accuracy: 94.33962264150944%
Accuracy: 90.9090909090909%
Accuracy: 91.83673469387756%
Average Accuracy:93.88981692162285%
k= 4
Accuracy: 89.79591836734694%
Accuracy: 94.11764705882352%
Accuracy: 94.54545454545455%
Accuracy: 88.13559322033898%
Accuracy: 100.0%
Average Accuracy:93.3189226383928%
k= 5
Accuracy: 96.29629629629629%
Accuracy: 94.11764705882352%
Accuracy: 89.0909090909091%
Accuracy: 96.15384615384616%
Accuracy: 98.21428571428571%
Average Accuracy:94.77459686283215%
k= 6
Accuracy: 93.10344827586206%
Accuracy: 94.64285714285714%
Accuracy: 96.29629629629629%
Accuracy: 92.98245614035088%
Acc